In [1]:
import numpy as np
import pandas as pd

In [28]:
p = 0.0
error = {}
for dataset in [1, 2]:
    error[dataset] = {}
    for fold_num in range(5):
        error[dataset][fold_num] = {}
        for appliance in ['hvac', 'fridge', 'dr', 'dw']:
            error[dataset][fold_num][appliance] = {}
            for cell_type in ['GRU', 'RNN', 'LSTM']:
                error[dataset][fold_num][appliance][cell_type] = {}
                for hidden_size  in [20, 50, 100]:
                    error[dataset][fold_num][appliance][cell_type][hidden_size] = {}
                    for num_layers in range(1,5):
                        error[dataset][fold_num][appliance][cell_type][hidden_size][num_layers] = {}
                        for bidirectional in [True, False]:
                            error[dataset][fold_num][appliance][cell_type][hidden_size][num_layers][bidirectional] = {}
                            for lr in [0.01, 0.1, 1.0]:
                                error[dataset][fold_num][appliance][cell_type][hidden_size][num_layers][bidirectional][lr] = {}
                                for iters in [1000, 2000, 3000]:
                                    error[dataset][fold_num][appliance][cell_type][hidden_size][num_layers][bidirectional][lr][iter] = \
                                    np.load('../../baseline/rnn-nested-cv/valid-error-{}-{}-{}-{}-{}-{}-{}-{}-{}-[\'{}\'].npy'.format(fold_num, \
                                                                                                                       dataset, 
                                                                                                                       cell_type, 
                                                                                                                       hidden_size, 
                                                                                                                       num_layers, 
                                                                                                                       bidirectional, 
                                                                                                                       lr, 
                                                                                                                       iters, 
                                                                                                                       p, appliance))

FileNotFoundError: [Errno 2] No such file or directory: "../../baseline/rnn-nested-cv/valid-error-3-1-RNN-20-1-True-0.1-1000-0.0-['hvac'].npy"